In [1]:
# 先生成一些脉冲序列

import numpy as np
import plotly.graph_objects as go

np.random.seed(1234)

total_time = 1000.0  
dt = 1.0          
t = np.arange(0, total_time + dt, dt) 
n_bins = len(t)

n_trials = 100
switch_time = total_time / 2  # 在 500 毫秒时切换条件
switch_bin = int(round(switch_time / dt))  # 对应的时间步索引

# 两种条件下的神经元发放率
rate_before_switch = 20.0  # 切换前的平均发放率（Hz）
rate_after_switch = 40.0   # 切换后的平均发放率（Hz）

# 创建刺激向量
stimulus = np.zeros(n_bins, dtype=int)
stimulus[switch_bin:] = 1

# 根据刺激条件计算每个时间步发生脉冲的概率
prob_spike = np.zeros(n_bins)
prob_spike[stimulus == 0] = rate_before_switch * (dt / 1000.0)  # 将 dt 转换为秒
prob_spike[stimulus == 1] = rate_after_switch * (dt / 1000.0)

# 模拟每次试验的脉冲序列
spike_trains = np.zeros((n_trials, n_bins), dtype=bool)
for trial in range(n_trials):
    spike_trains[trial, :] = np.random.rand(n_bins) < prob_spike

raster_x = []
raster_y = []
for trial in range(n_trials):
    spike_times = t[spike_trains[trial, :]]
    raster_x.extend(spike_times)
    raster_y.extend([trial + 1] * len(spike_times))  

In [4]:
from plotly.subplots import make_subplots

# 可视化：光栅图和平滑后的 PSTH（每秒脉冲直方图）。这里使用高斯核进行平滑。
spike_times = []
trial_numbers = []

for trial in range(n_trials):
    trial_spike_indices = np.where(spike_trains[trial, :])[0]
    trial_spike_times = t[trial_spike_indices]
    spike_times.extend(trial_spike_times)
    trial_numbers.extend([trial + 1] * len(trial_spike_times)) 

# 计算 PSTH（每个时间步的发放脉冲计数）
psth = np.sum(spike_trains, axis=0)

# 将 PSTH 归一化为发放率（单位 Hz）
psth_rate = psth / n_trials / (dt / 1000.0) 

# 定义高斯核函数
def gaussian_kernel(sigma, dt):
    size = int(round(6 * sigma / dt))  
    if size % 2 == 0:
        size += 1  # 确保核大小为奇数
    x = np.linspace(-(size // 2), size // 2, size)  
    kernel = np.exp(-0.5 * (x / (sigma / dt)) ** 2)  
    kernel /= np.sum(kernel)  
    return kernel

kernel_sigma = 50.0  
kernel = gaussian_kernel(kernel_sigma, dt)  
smoothed_rate = np.convolve(psth_rate, kernel, mode="same")  

fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,  
    vertical_spacing=0.1,  
    subplot_titles=("Raster Plot", "PSTH with Smoothed Firing Rate")
)

fig.add_trace(go.Scatter(
    x=spike_times,
    y=trial_numbers,
    mode="markers",
    marker=dict(size=3, color="blue"),
    name="Raster Plot"
), row=1, col=1)
fig.add_trace(go.Bar(
    x=t,
    y=psth_rate,
    name="PSTH",
    marker=dict(color="lightblue"),
    opacity=0.5
), row=2, col=1)
fig.add_trace(go.Scatter(
    x=t,
    y=smoothed_rate,
    mode="lines",
    line=dict(color="red", width=2),
    name="Smoothed Firing Rate"
), row=2, col=1)
fig.add_shape(
    dict(
        type="line",
        x0=switch_time,
        x1=switch_time,
        y0=0,
        y1=n_trials,
        line=dict(color="green", dash="dash"),
    ),
    row=1, col=1
)

fig.add_shape(
    dict(
        type="line",
        x0=switch_time,
        x1=switch_time,
        y0=0,
        y1=max(psth_rate),
        line=dict(color="green", dash="dash"),
    ),
    row=2, col=1
)
fig.update_layout(
    height=700,
    title="Spike Raster Plot and PSTH with Smoothed Firing Rate",
    xaxis=dict(title="Time (ms)"),
    yaxis=dict(title="Trial Number"),
    xaxis2=dict(title="Time (ms)"),
    yaxis2=dict(title="Firing Rate (Hz)"),
    showlegend=True
)
fig.show()

In [5]:
# 计算 shannon information
p_spike = np.mean(spike_trains)  # 所有试验中脉冲发生的概率
p_no_spike = 1.0 - p_spike  # 脉冲未发生的概率

# 定义计算香农熵的函数
def shannon_entropy(p):
    if p == 0.0 or p == 1.0:
        return 0.0  
    else:
        return -p * np.log2(p) - (1.0 - p) * np.log2(1.0 - p)

entropy = shannon_entropy(p_spike)
print(f"Probability of spike per bin (across all trials): {p_spike:.4f}")
print(f"Shannon entropy of spike trains: {entropy:.4f} bits")

Probability of spike per bin (across all trials): 0.0296
Shannon entropy of spike trains: 0.1925 bits


计算互信息

In [6]:
total_observations = n_trials * n_bins

# 计算联合概率 P(S, R)
# 初始化计数矩阵
counts = np.zeros((2, 2), dtype=int)

for trial in range(n_trials):
    for bin in range(n_bins):
        s = stimulus[bin]  # S=0 or 1
        r = 1 if spike_trains[trial, bin] else 0  # R=0 or 1
        counts[s, r] += 1  # 更新计数

# 将计数转换为概率
P_SR = counts / total_observations 
P_S = np.sum(P_SR, axis=1)  
P_R = np.sum(P_SR, axis=0)  

# 计算互信息 I(S;R)
MI = 0.0
for s in range(2):  
    for r in range(2):  
        P_s_r = P_SR[s, r]
        P_s = P_S[s]
        P_r = P_R[r]
        if P_s_r > 0.0:  
            MI += P_s_r * np.log2(P_s_r / (P_s * P_r))

# 输出结果
print(f"Mutual Information between stimulus and response: {MI:.4f} bits")

Mutual Information between stimulus and response: 0.0023 bits
